# Interesting Points
This notebook reivews the ability to generate interesting points from processed imagery

### Import some libraries, configure Django

In [1]:
import os
import django
import pandas as pd
import geopandas as gpd
import subprocess
from shapely.wkt import loads
from asgiref.sync import sync_to_async
from django.core.management import call_command
from django.contrib.gis.geos import GEOSGeometry

import sys; sys.path.append('../../')
os.environ['DJANGO_SETTINGS_MODULE'] = 'gaia.settings'
django.setup()

from whale.models import ExtractTransformLoad as ETL
from whale.models import PointsOfInterest as POI

### User defined variables

In [8]:
interesting_points = 'C:/gis/apps/libs/generate_interesting_points.py'
input_url = "C:/gis/apps/data/21APR24154029-S1BS-506967347100_01_P001_u08mr32619.tif"

poi_columns = ['id', 'catalog_id', 'vendor_id', 'entity_id', 'sample_idx', 'area', 'deviation', 'epsg_code']

### Generate Interesting Points
This outputs GeoJSON

In [3]:
output_fn = input_url.replace('tif', 'geojson')

subprocess.run([sys.executable, interesting_points, '--input_url', input_url,
                '--output_fn', output_fn, '--method', 'big_window',
                '--difference_threshold', '20', '--overwrite'])

CompletedProcess(args=['C:\\Users\\john.wall\\AppData\\Local\\anaconda3\\envs\\gaia\\python.exe', 'C:/gis/apps/libs/generate_interesting_points.py', '--input_url', 'C:/gis/apps/data/21APR24154029-S1BS-506967347100_01_P001_u08mr32619.tif', '--output_fn', 'C:/gis/apps/data/21APR24154029-S1BS-506967347100_01_P001_u08mr32619.geojson', '--method', 'big_window', '--difference_threshold', '20', '--overwrite'], returncode=0)

### Create Shapefile from GeoJSON for headsup review

In [4]:
gdf = gpd.read_file(output_fn)
output_shp = output_fn.replace('geojson', 'shp')
gdf.to_file(output_shp)

### Review Interesting Points GeoJSON

In [5]:
gdf = gpd.read_file(output_fn)
print(f"The shape of your Geodataframe is: {gdf.shape}\n")
gdf.head()

The shape of your Geodataframe is: (63, 4)



,id,area,deviation,geometry
0,2,1451.594751,13.576925,POINT (389007.941 4630669.296)
1,3,2672.599184,10.939635,POINT (388548.646 4630661.489)
2,30,2.870419,89.703911,POINT (398380.273 4630377.041)
3,67,4.784031,33.603260,POINT (390573.116 4630166.341)
4,89,3.348822,61.378738,POINT (397539.492 4630018.522)


### Add Interesting Points to SpatiaLite Database

In [6]:
def import_poi(geojson_path):
    """ Synchronous Import Points of Interest function.

        Takes the GeoJSON filepath and converts the file path to Vendor ID
            using the panchromatic image as the basis for this (opposed to
            the multispectral). Queries the ExtractTransformLoad (ETL) table
            in SpatiaLite for the relevant Vendor ID object. Reads the GeoJSON
            file to a GeoDataFrame. Updates or creates the Interesting Points
            records from a combination of the ETL and GeoJSON information.

        Print statements support troubleshooting.
    """
    vid = '_'.join(geojson_path.split('/')[-1:][0].split('.')[0].split('_')[:-1]).replace('S1BS', 'P1BS')
    obj = ETL.objects.get(vendor_id=vid)

    epsg_code = geojson_path.split('/')[-1:][0].split('.')[0].split('_')[-1:][0].split('mr')[-1:][0]
    gdf = gpd.read_file(geojson_path)

    for index, row in gdf.iterrows():
        print(f"Processing row: {row['id']}")
        poi, created = POI.objects.update_or_create(
            sample_idx = row['id'],
            defaults={
                'catalog_id': obj.id,
                'vendor_id': obj.vendor_id,
                'entity_id': obj.entity_id,
                'area': row['area'],
                'deviation': row['deviation'],
                'epsg_code': epsg_code,
                'point': row['geometry'].wkt
            }
        )
        print(f"\t{'Created' if created else 'Updated'} POI with id: {poi.sample_idx}\n")

    print('Data imported successfully!')

async def import_poi_async(file_path):
    await sync_to_async(import_poi, thread_sensitive=True)(file_path)

import asyncio

if asyncio.get_event_loop().is_running():
    await import_poi_async(output_fn)
else:
    asyncio.run(import_poi_async(output_fn))

Processing row: 2
	Updated POI with id: 2

Processing row: 3
	Updated POI with id: 3

Processing row: 30
	Updated POI with id: 30

Processing row: 67
	Updated POI with id: 67

Processing row: 89
	Updated POI with id: 89

Processing row: 101
	Created POI with id: 101

Processing row: 112
	Updated POI with id: 112

Processing row: 117
	Updated POI with id: 117

Processing row: 122
	Updated POI with id: 122

Processing row: 123
	Created POI with id: 123

Processing row: 133
	Created POI with id: 133

Processing row: 143
	Updated POI with id: 143

Processing row: 154
	Updated POI with id: 154

Processing row: 160
	Created POI with id: 160

Processing row: 183
	Updated POI with id: 183

Processing row: 204
	Created POI with id: 204

Processing row: 218
	Created POI with id: 218

Processing row: 250
	Updated POI with id: 250

Processing row: 294
	Updated POI with id: 294

Processing row: 303
	Updated POI with id: 303

Processing row: 342
	Created POI with id: 342

Processing row: 344
	Update

### Confirm that the points were added

In [9]:
objs = await sync_to_async(list)(POI.objects.all())
print(f"Number of POI records in database: {len(objs)}\n")

vid = '_'.join(output_fn.split('/')[-1:][0].split('.')[0].split('_')[:-1]).replace('S1BS', 'P1BS')

geoms = []
attributes = []
for obj in objs:
    if obj.vendor_id == vid:
        attr_dict = {col: getattr(obj, col) for col in poi_columns}
        attributes.append(attr_dict)
    
        geoms.append(GEOSGeometry(obj.point))

gdf = gpd.GeoDataFrame(attributes, geometry = [loads(g.wkt) for g in geoms])
gdf.head()

Number of POI records in database: 108426



,id,catalog_id,vendor_id,entity_id,sample_idx,area,deviation,epsg_code,geometry
0,3,1040010067D36B00,21APR24154029-P1BS-506967347100_01_P001,WV320210424154029P01,30,2.870419,89.703911,32619,POINT (398380.273 4630377.041)
1,17,1040010067D36B00,21APR24154029-P1BS-506967347100_01_P001,WV320210424154029P01,122,2.870419,15.009857,32619,POINT (390992.942 4629695.776)
2,147,1040010067D36B00,21APR24154029-P1BS-506967347100_01_P001,WV320210424154029P01,67,4.784031,33.603260,32619,POINT (390573.116 4630166.341)
3,156,1040010067D36B00,21APR24154029-P1BS-506967347100_01_P001,WV320210424154029P01,154,2.870419,87.689003,32619,POINT (398785.638 4629443.547)
4,190,1040010067D36B00,21APR24154029-P1BS-506967347100_01_P001,WV320210424154029P01,443,3.109620,72.580132,32619,POINT (395215.550 4625337.131)


### Identify unique Vendor IDs

In [10]:
objs = await sync_to_async(list)(POI.objects.all())
print(f"Number of POI records in database: {len(objs)}\n")

vendor_ids = list(set([obj.vendor_id for obj in objs]))
print(f"Your unique vendor ids are: {vendor_ids}")

Number of POI records in database: 41775

Your unique vendor ids are: ['24FEB11151605-P1BS-508192289030_03_P002', '24MAR01151632-P1BS-508221188010_01_P004', '23OCT21153005-P1BS-507904384010_01_P001', '24MAR01151633-P1BS-508221188010_01_P005', '24MAY10151201-P1BS-508496072030_01_P003', '23JUL11151855-P1BS-507622921010_01_P003', '23OCT08152539-P1BS-507875362010_02_P004', '24MAY10151200-P1BS-508496072030_01_P002']


In [17]:
vendor_ids_list = [obj.vendor_id for obj in objs]
vendor_dict = {}
for vendor_id in vendor_ids:
    vendor_dict.update({
        vendor_id: vendor_ids_list.count(vendor_id)
    })

pd.DataFrame.from_dict(vendor_dict, orient='index', columns=['poi'])

,poi
24FEB11151605-P1BS-508192289030_03_P002,1299
24MAR01151632-P1BS-508221188010_01_P004,54
23OCT21153005-P1BS-507904384010_01_P001,8
24MAR01151633-P1BS-508221188010_01_P005,4
24MAY10151201-P1BS-508496072030_01_P003,871
23JUL11151855-P1BS-507622921010_01_P003,4810
23OCT08152539-P1BS-507875362010_02_P004,34407
24MAY10151200-P1BS-508496072030_01_P002,322


In [14]:
objs = await sync_to_async(list)(POI.objects.all())
print(f"Number of POI records in database: {len(objs)}\n")

vid = "21APR24154032-P1BS-506967347100_01_P004"

geoms = []
attributes = []
for obj in objs:
    if obj.vendor_id == vid:
        attr_dict = {col: getattr(obj, col) for col in poi_columns}
        attributes.append(attr_dict)
    
        geoms.append(GEOSGeometry(obj.point))

gdf = gpd.GeoDataFrame(attributes, geometry = [loads(g.wkt) for g in geoms])
gdf.head()

Number of POI records in database: 108405



,geometry


# End